# Text Preprocessing:

Data cleaning is an important and intensive process in Data science which aids in data analysis and building machine learning models.

In [1071]:
import pandas as pd
import unidecode 
import re
import time 
import stopwords 
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk import word_tokenize
import gensim
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
import langid

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [1072]:
def get_data_from_csv(url):
    df = pd.read_csv(url)
    return df

# Now we start preparing the data:

- Lowercase the text
- Remove links
- Remove non english language
- remove punctuation, stop words

https://github.com/kk7nc/Text_Classification

# Stop words
Text and document classification over social media, such as Twitter, Facebook, and so on is usually affected by the noisy nature (abbreviations, irregular forms) of the text corpuses.


# Capitalization
Sentences can contain a mixture of uppercase and lower case letters. Multiple sentences make up a text document. To reduce the problem space, the most common approach is to reduce everything to lower case. This brings all words in a document in same space, but it often changes the meaning of some words, such as "US" to "us" where first one represents the United States of America and second one is a pronoun. To solve this, slang and abbreviation converters can be applied.

# Noise Removal
Another issue of text cleaning as a pre-processing step is noise removal. Text documents generally contains characters like punctuations or special characters and they are not necessary for text mining or classification purposes. Although punctuation is critical to understand the meaning of the sentence, but it can affect the classification algorithms negatively.

In [1073]:
def cleaning_preprocessing_data_from_csv(data):  
    cleaned_data = []
    
    for text in data:

        # Replacing all the occurrences of \n,\\n,\t,\\ with a space.
        formatted_text = text.replace('\\n', ' ').replace('\n', ' ').replace('\t',' ').replace('\\', ' ').replace('. com', '.com')

        # Removing all the occurrences of links that starts with https
        formatted_text = re.sub(r'http\S+', '', formatted_text)

        # Remove all the occurrences of text that ends with .com
        formatted_text = re.sub(r"\ [A-Za-z]*\.com", " ", formatted_text)

        # Remove all whitespaces
        pattern = re.compile(r'\s+') 
        formatted_text = re.sub(pattern, ' ', formatted_text)
        formatted_text = formatted_text.replace('?', ' ? ').replace(')', ') ')

        # Remove accented characters from text using unidecode.
        # Unidecode() - It takes unicode data & tries to represent it to ASCII characters. 
        remove_character = unidecode.unidecode(formatted_text)

        # Convert text to lower case
        lower_text = remove_character.lower()

        # Pattern matching for all case alphabets
        Pattern_alpha = re.compile(r"([A-Za-z])\1{1,}", re.DOTALL)

        # Limiting all the  repeatation to two characters.
        formatted_text = Pattern_alpha.sub(r"\1\1", lower_text) 

        # Pattern matching for all the punctuations that can occur
        Pattern_Punct = re.compile(r'(\'[.,/#!"$<>@[]^&%^&*?;:{}=_`~()+-])\1{1,}')

        # Limiting punctuations in previously formatted string to only one.
        Combined_Formatted = Pattern_Punct.sub(r'\1', formatted_text)

        # The below statement is replacing repeatation of spaces that occur more than two times with that of one occurrence.
        Final_Formatted = re.sub(' {2,}',' ', Combined_Formatted)

        # The formatted text after removing not necessary punctuations.
        Formatted_Text = re.sub(r"[^a-zA-Z]+", ' ', Final_Formatted) 

        # Text without stopwords
        remove_stop_words = repr(Formatted_Text)
        stoplist = stopwords.words('english') 
        
        # Append words to Medium.com
        stoplist.extend(['ago', 'followers', 'pinned', 'read', 'min', 'published', 'days', 'hours'])
            
        No_StopWords = [word for word in word_tokenize(remove_stop_words) if word.lower() not in stoplist ]

        # Convert list of tokens_without_stopwords to String type.
        words_string = ' '.join(No_StopWords) 

        # Remove more stop words  
        final_text = remove_stopwords(words_string) 
        
        # Split the "'" from the edges
        cleaned_data.append(final_text[1:len(final_text)-1])
        
    return cleaned_data

In [1074]:
def remove_non_english_articles_and_remove_duplicated_rows(df):
    # Remove duplicated rows
    df.drop_duplicates(inplace=True)
    
    for topic in df["topic"]:   
        DetectorFactory.seed = 0
        if detect(topic) != "en":
            print("Found different language: " + detect(topic))
            df.drop(df.index[(df["topic"] == topic)], axis=0, inplace=True)
            
    for content in df["content"]:
        DetectorFactory.seed = 0
        if detect(content) != "en":
            print("Found different language: " + detect(content))
            df.drop(df.index[(df["content"] == content)], axis=0, inplace=True)        

    return df

In [1075]:
def insert_df_to_csv(topic, category, content, csv_file_name):
    data = pd.DataFrame({
    "topic": topic,
    "category": category,
    "content": content
    })
    data.to_csv(csv_file_name)

In [1076]:
df = get_data_from_csv("example.csv")
print("Before cleaning data the shape of data frame is : " + str(df.shape))
df

Before cleaning data the shape of data frame is : (24, 4)


,Unnamed: 0,topic,category,content
0,0,Things You Learn As An Investor,business,29 Followers Pinned NFTs or non-fungible toke...
1,1,Even the Best Startup Leaders Have This Weakne...,business,19.1K Followers Pinned All the actionable stu...
2,2,Econ Made Me Jack Welch All My Friends,business,32 Followers 16 hours ago Or: How I learned t...
3,3,10 Skills That’ll Make You Money (In 2022),business,10.6K Followers Pinned How To Learn Faster By...
4,4,Your Biggest Risk Is You,business,647 Followers Published in Towards Data Scien...
5,5,“Scope 3” is a Poison Pill,business,Pinned Envisioning a GHG protocol that inclu...
6,6,What is happening to Nasdaq and FAANG?,business,24 Followers 1 day ago Since the Great Financ...
7,7,The Importance of Auditing Your Attention: How...,business,1.3K Followers Pinned One of the most challen...
8,8,PR Pros: Mira Marcus On The 5 Things You Need ...,business,8 Followers Published in Authority Magazine ·...
9,9,Why Some Emails Have 40% Open Rates and Others...,business,108K Followers Published in Change Your Mind ...


In [1077]:
df = remove_non_english_articles_and_remove_duplicated_rows(df)
title_list = cleaning_preprocessing_data_from_csv(df["topic"])
content_list = cleaning_preprocessing_data_from_csv(df["content"])

insert_df_to_csv(title_list, df["category"], content_list, "cleaned-scraping-data.csv")
print("After cleaning data the shape of data frame is : " + str(df.shape))
df

Found different language: de
Found different language: no
Found different language: nl
After cleaning data the shape of data frame is : (21, 4)


,Unnamed: 0,topic,category,content
0,0,Things You Learn As An Investor,business,29 Followers Pinned NFTs or non-fungible toke...
1,1,Even the Best Startup Leaders Have This Weakne...,business,19.1K Followers Pinned All the actionable stu...
3,3,10 Skills That’ll Make You Money (In 2022),business,10.6K Followers Pinned How To Learn Faster By...
5,5,“Scope 3” is a Poison Pill,business,Pinned Envisioning a GHG protocol that inclu...
6,6,What is happening to Nasdaq and FAANG?,business,24 Followers 1 day ago Since the Great Financ...
7,7,The Importance of Auditing Your Attention: How...,business,1.3K Followers Pinned One of the most challen...
8,8,PR Pros: Mira Marcus On The 5 Things You Need ...,business,8 Followers Published in Authority Magazine ·...
9,9,Why Some Emails Have 40% Open Rates and Others...,business,108K Followers Published in Change Your Mind ...
10,0,A Question for Moderate Republicans,politics,I have been voting for a very long time. Some...
12,2,Politics at Trinity High School,politics,Politics at Trinity High School Those who kno...
